Cài docker desktop

Mở docker desktop

chạy trong thư mục đang làm việc 2 lệnh sau:
- docker pull qdrant/qdrant
- docker run -p 6333:6333 qdrant/qdrant

mở localhost:6333 để set api key

chèn api key vào script bên dưới (chỗ compute similarity)

chạy script bên dưới

In [1]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio
! pip install ipywidgets
! pip install qdrant-client
! pip install httpcore==0.17.3
! pip install httpx==0.24.1
! pip install h11==0.14.0


In [2]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

In [3]:
# load dataset
dataset = fo.Dataset.from_images_dir('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes', name=None, tags=None, recursive=True)

 100% |█████████████| 87306/87306 [5.9s elapsed, 0s remaining, 14.8K samples/s]      


### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [4]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [5]:
for sample in dataset:
    object_path = f"/Users/duyennguyen/Downloads/AIC/Batch_01/objects/{sample['video']}/{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
        
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()

### Thêm thông tin CLIP embedding.

In [6]:
all_keyframe = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('/Users/duyennguyen/Downloads/AIC/Batch_01/Keyframes/*/keyframes-*/*')
all_video = [v.rsplit('/',1)[-1] for v in all_video]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [7]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [8]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [9]:
embedding_dict = {}
for v in all_video:
    clip_path = f'/Users/duyennguyen/Downloads/AIC/Batch_01/clip-features-vit-b32/{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]

Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [10]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [11]:
# # Clean up    
# # Delete run record from FiftyOne
# # Only run this if you get the error "brainkey 'qdrant_index' already exists"
# dataset.delete_brain_run("qdrant_index") # milvus_index, img_sim, tmp

In [12]:
qdrant_index = fob.compute_similarity(
    # qdrant
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,         # precomputed image embeddings
    backend="qdrant",                  # sklearn, qdrant, pinecone, milvus, lancedb
    brain_key="qdrant_index",                # img_sim, tmp, milvus_index, lancedb_index
    metric="cosine",
    replication_factor=1,
    shard_number=1,
    url="http://localhost:6333",
    api_key="f0874247-fbc6-4d22-8752-2b9014564484",
    grpc_port=6334,
    prefer_grpc=False
)

/Users/duyennguyen/anaconda3/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:115: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [13]:
# dataset.export(
#     './qdrant_index_121023',
#     dataset_type=fo.types.FiftyOneDataset
#     )

In [14]:
print(dataset.get_brain_info('qdrant_index'))
qdrant_index = dataset.load_brain_results('qdrant_index')

{
    "key": "qdrant_index",
    "version": "0.21.4",
    "timestamp": "2023-10-14T06:51:50.542753",
    "config": {
        "method": "qdrant",
        "cls": "fiftyone.brain.internal.core.qdrant.QdrantSimilarityConfig",
        "embeddings_field": null,
        "model": "clip-vit-base32-torch",
        "patches_field": null,
        "supports_prompts": true,
        "collection_name": "fiftyone-2023-10-14-13-48-45",
        "metric": "cosine",
        "replication_factor": 1,
        "shard_number": 1,
        "write_consistency_factor": null,
        "hnsw_config": null,
        "optimizers_config": null,
        "wal_config": null
    }
}


In [15]:
print(fob.brain_config)

{
    "default_similarity_backend": "sklearn",
    "similarity_backends": {
        "lancedb": {
            "config_cls": "fiftyone.brain.internal.core.lancedb.LanceDBSimilarityConfig"
        },
        "milvus": {
            "config_cls": "fiftyone.brain.internal.core.milvus.MilvusSimilarityConfig"
        },
        "pinecone": {
            "config_cls": "fiftyone.brain.internal.core.pinecone.PineconeSimilarityConfig"
        },
        "qdrant": {
            "config_cls": "fiftyone.brain.internal.core.qdrant.QdrantSimilarityConfig"
        },
        "sklearn": {
            "config_cls": "fiftyone.brain.internal.core.sklearn.SklearnSimilarityConfig"
        }
    }
}


In [16]:
_id = dataset.first().id
print(type(_id))


<class 'str'>


In [17]:
# Step 4: Query your data
query = dataset.first().id  # query by sample ID
view = dataset.sort_by_similarity(
    query,
    brain_key="qdrant_index",
    k=10,  # limit to 10 most similar samples
)

# Step 5 (optional): Cleanup

# # Delete the qdrant table
# lancedb_index.cleanup()

# # Delete run record from FiftyOne
# dataset.delete_brain_run("lancedb_index")

In [18]:
session = fo.launch_app(dataset, port =5152)
session.view = view

In [19]:
# Query text
query = "man driving car"

# Perform a text query
view = dataset.sort_by_similarity(
    query, 
    k=15, 
    brain_key="qdrant_index"
)

session.view = view